In [ ]:
import pandas as pd
# This is correct 1.
def difference_between_moving_averages(df, short_term_window = 6, long_term_window = 24, column = 'Adj Close'):
    """
    This function computes the difference between a short-term and long-term moving average
    of a specified column in the given DataFrame.

    Parameters:
    df (pd.DataFrame): Input DataFrame with a 'date' column as the index
    column (str): The column name to compute the moving averages on
    short_term_window (int): The window size for the short-term moving average
    long_term_window (int): The window size for the long-term moving average

    Returns:
    pd.DataFrame: DataFrame with the date as the index and the difference of the moving averages as the values
    """
    
    # Ensure the dataframe's index is datetime
    if not pd.api.types.is_datetime64_any_dtype(df.index):
        df['date'] = pd.to_datetime(df['date'])
        df.set_index('date', inplace=True)
    
    # Calculate short-term and long-term moving averages
    short_term_ma = df[column].rolling(window=short_term_window).mean()
    long_term_ma = df[column].rolling(window=long_term_window).mean()
    
    # Calculate the difference between short-term and long-term moving averages
    difference = short_term_ma - long_term_ma
    
    difference = pd.DataFrame(difference)
    difference.columns = ['MA_Diff']
    return difference



In [ ]:
import pandas as pd # This is correct 2.

def moving_average_difference(df, lookback_period = 24, col = "Adj Close"):
    # Ensure the DataFrame has a valid index (dates)
    '''
    This function return the difference between a price series's current price and its rolling mean
    '''
    if not isinstance(df.index, pd.DatetimeIndex):
        raise ValueError("The DataFrame index should be of type pd.DatetimeIndex.")
    
    # Calculate the rolling (moving) mean over the lookback period
    rolling_mean = df[col].rolling(window=lookback_period).mean()
    
    # Calculate the difference between the current price and the moving average
    difference = df[col] - rolling_mean
    
    # Return a DataFrame with the date index and the calculated difference
    result = pd.DataFrame({
        'Price': df[col],
        'Rolling_Mean': rolling_mean,
        'Difference': difference
    })
    
    return result.dropna()  # Drop the first 'lookback_period - 1' rows that have NaN values

In [ ]:
# This is correct 3.
import pandas as pd

def second_derivative(df, price_column='Adj Close'):
    '''
    Return both the first and second derivative of the price series, can use both as factors
    '''
    
    # Ensure the DataFrame has a valid index (dates)
    if not isinstance(df.index, pd.DatetimeIndex):
        raise ValueError("The DataFrame index should be of type pd.DatetimeIndex.")
    
    # Ensure the column exists in the DataFrame
    if price_column not in df.columns:
        raise ValueError(f"'{price_column}' column not found in the DataFrame.")
    
    # Calculate the first derivative (daily returns as percentage change)
    first_derivative = df[price_column].pct_change().dropna()
    
    # Calculate the second derivative (difference of consecutive first derivatives)
    second_derivative = first_derivative.diff().dropna()
    
    # Create a DataFrame with the results, preserving the date index
    result = pd.DataFrame({
        'Price': df[price_column],
        'First_Derivative': first_derivative,
        'Second_Derivative': second_derivative
    })
    
    return result

In [ ]:
# This is correct 4. 

import numpy as np
import pandas as pd

def rolling_volatility(df, lookback_period = 24, price_column='Adj Close'):
    '''
    Return rolling volatility of price over a period
    '''
    
    # Ensure the DataFrame has a valid index (dates)
    if not isinstance(df.index, pd.DatetimeIndex):
        raise ValueError("The DataFrame index should be of type pd.DatetimeIndex.")
    
    # Ensure the column exists in the DataFrame
    if price_column not in df.columns:
        raise ValueError(f"'{price_column}' column not found in the DataFrame.")
    
    # Calculate daily returns as percentage change between consecutive prices
    daily_returns = df[price_column].pct_change().dropna()
    
    # Calculate rolling standard deviation (volatility) over the lookback period
    rolling_volatility = daily_returns.rolling(window=lookback_period).std() * np.sqrt(252)  # Annualized volatility
    
    # Return the rolling volatility as a DataFrame
    result_df = pd.DataFrame({
        'Price': df[price_column],
        'Rolling_Volatility': rolling_volatility
    })
    
    return result_df

In [ ]:
import pandas as pd #### Correct 5.

def rolling_mean_difference(df, lookback_period, price_column='Adj Close'):
    '''
    This function returns the difference in the count of days the price of a stock is above its moving average and subtracts
    from the amount of days it is below its moving average
    '''

    
    # Ensure the DataFrame has a valid index (dates)
    if not isinstance(df.index, pd.DatetimeIndex):
        raise ValueError("The DataFrame index should be of type pd.DatetimeIndex.")
    
    # Calculate the rolling mean (this will be NaN for the first lookback_period-1 rows)
    rolling_mean = df[price_column].rolling(window=lookback_period).mean()
    
    # Initialize an empty list to store the rolling differences (above - below)
    rolling_difference = [None] * (lookback_period )  # Start with None for the first `lookback_period-1` rows

    # Iterate through the stock prices starting from the lookback_period-th element
    for i in range(lookback_period, len(df)):
        # Get the current rolling window (the last `lookback_period` elements)
        current_window = df[price_column].iloc[i-lookback_period:i]
        
        # Calculate the rolling mean for the current window
        running_mean = current_window.mean()
        
        # Count how many days are above and below the running mean in the window
        count_above = (current_window > running_mean).sum()
        count_below = (current_window < running_mean).sum()
        
        # Store the difference (days above - days below)
        rolling_difference.append(count_above - count_below)
    print(len(df))
    print(len(rolling_mean))
    print(len(rolling_difference))    
    # Now the rolling_difference list should have the correct length, matching df's length
    result_df = pd.DataFrame({
        'Price': df[price_column],
        'Rolling_Mean': rolling_mean,
        'Rolling_Difference': rolling_difference  # This list now matches the length of df
    })


    return result_df
